In [1]:
import logging
from pathlib import Path

import numpy as np
import pandas as pd

from src.idspy.data.split import random_split
from src.idspy.data.tabular_data import TabularData, TabularSchema
from src.idspy.data.tabular_repository import TabularDataRepository
from src.idspy.services.logger import setup_logging

setup_logging()
logger = logging.getLogger(__name__)

In [2]:
rng = np.random.default_rng(42)
n = 12
df = pd.DataFrame({
    "id": range(1, n + 1),
    "age": rng.integers(18, 70, size=n),
    "income": rng.normal(30000, 8000, size=n).round(0),
    "city": rng.choice(["Roma", "Milano", "Torino"], size=n),
    "label": rng.choice([0, 1], size=n, p=[0.6, 0.4]),
})
df.set_index("id", inplace=True)

In [3]:
schema = TabularSchema(
    target="label",
    numeric=("age", "income"),
    categorical=("city",),
)

In [4]:
tab = TabularData(_base=df, _schema=schema)

logger.info(f"numeric:\n{tab.numeric}")
logger.info(f"categorical:\n{tab.categorical}")

INFO     18:49:56: numeric:                                                                                        
             age   income                                                                                          
         id                                                                                                        
         1    22  31023.0                                                                                          
         2    58  27470.0                                                                                          
         3    52  29866.0                                                                                          
         4    40  23176.0                                                                                          
         5    40  37035.0                                                                                          
         6    62  36222.0                                                                                          
         7    22  30528.0                                                                                          
         8    54  39018.0                                                                                          
         9    28  33740.0                                                                                          
         10   22  23126.0                                                                                          
         11   45  32950.0                                                                                          
         12   68  22329.0

INFO     18:49:56: categorical:                                                                                    
               city                                                                                                
         id                                                                                                        
         1   Torino                                                                                                
         2   Torino                                                                                                
         3     Roma                                                                                                
         4   Milano                                                                                                
         5     Roma                                                                                                
         6   Torino                                                                                                
         7   Torino                                                                                                
         8   Milano                                                                                                
         9     Roma                                                                                                
         10  Torino                                                                                                
         11  Milano                                                                                                
         12  Torino

In [5]:
num = tab.numeric
num["age"] = num["age"] + 2
num["income"] = (num["income"] * 1.05).round(0)
tab.numeric = num

cat = tab.categorical
cat["city"] = cat["city"].replace({"Milano": "MILANO"})
tab.categorical = cat

logger.info(f"features:\n{tab.features}")

INFO     18:49:56: features:                                                                                       
             age   income    city                                                                                  
         id                                                                                                        
         1    24  32574.0  Torino                                                                                  
         2    60  28844.0  Torino                                                                                  
         3    54  31359.0    Roma                                                                                  
         4    42  24335.0  MILANO                                                                                  
         5    42  38887.0    Roma                                                                                  
         6    64  38033.0  Torino                                                                                  
         7    24  32054.0  Torino                                                                                  
         8    56  40969.0  MILANO                                                                                  
         9    30  35427.0    Roma                                                                                  
         10   24  24282.0  Torino                                                                                  
         11   47  34598.0  MILANO                                                                                  
         12   70  23445.0  Torino

In [6]:
tab_view = tab.view_from_query("city == 'MILANO' and age > 50")

logger.info(f"features:\n{tab_view.features}")

INFO     18:49:56: features:                                                                                       
             age   income    city                                                                                  
         id                                                                                                        
         8    56  40969.0  MILANO

In [7]:
tab_view1 = tab.view_from_query("city == 'MILANO'")
tab_view2 = tab.view_from_query("age > 50")

tab_view = tab_view1.intersect(tab_view2)
logger.info(f"features:\n{tab_view.features}")

INFO     18:49:56: features:                                                                                       
             age   income    city                                                                                  
         id                                                                                                        
         8    56  40969.0  MILANO

In [8]:
cat = tab_view.categorical
cat["city"] = cat["city"].replace({"MILANO": "Milano"})
tab_view.categorical = cat

logger.info(f"features:\n{tab_view.features}")

INFO     18:49:56: features:                                                                                       
             age   income    city                                                                                  
         id                                                                                                        
         8    56  40969.0  Milano

In [9]:
train_idx, val_idx, test_idx = random_split(
    tab.data, train_size=0.7, val_size=0.15, test_size=0.15, random_state=123
)

train_view = tab.view(train_idx)
val_view = tab.view(val_idx)
test_view = tab.view(test_idx)

logger.info(f"train size: {len(train_view)}, val size: {len(val_view)}, test size: {len(test_view)}")

INFO     18:49:56: train size: 8, val size: 2, test size: 2

In [10]:
train_tab = train_view.materialize()
assert isinstance(train_tab, TabularData)

In [11]:
out_path = Path("example_dataset.parquet")
TabularDataRepository.save(tab, out_path, include_schema=True, index=True)
tab_loaded = TabularDataRepository.load(out_path)

logger.info(f"schema: {tab_loaded.schema.to_dict()}")
logger.info(f"data:\n{tab_loaded.data.head()}")

INFO     18:49:56: schema: {'target': 'label', 'numeric': ['age', 'income'], 'categorical': ['city'], 'extra': []}

INFO     18:49:56: data:                                                                                           
            id  age   income    city  label                                                                        
         0   1   24  32574.0  Torino      1                                                                        
         1   2   60  28844.0  Torino      0                                                                        
         2   3   54  31359.0    Roma      0                                                                        
         3   4   42  24335.0  MILANO      0                                                                        
         4   5   42  38887.0    Roma      0